In [ ]:
import os
import time
import numpy as np
import cv2
import tifffile
import yaml
from pydicom import dcmread
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import tkinter as tk
from tkinter import Label, messagebox, Entry, Button, filedialog
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Load configuration from YAML file
with open("config.yml", "r") as config_file:
    config = yaml.safe_load(config_file)

# Global thresholds from config file
MSE_THRESHOLD = config['thresholds']['mse_threshold']
LAPLACIAN_VAR_THRESHOLD = config['thresholds']['laplacian_var_threshold']
FOLDER_PATH = config['application']['folder_path']

# ImageProcessor class handles MSE and Laplacian variance computations
class ImageProcessor:
    @staticmethod
    def compute_mse(image1, image2):
        if image1.shape != image2.shape:
            raise ValueError("Images must have the same dimensions for MSE computation.")
        mse = np.mean((image1 - image2) ** 2)
        return mse

    @staticmethod
    def compute_laplacian_variance(image):
        laplacian = cv2.Laplacian(image, cv2.CV_64F)
        variance = laplacian.var()
        return variance

# Function to convert DICOM to TIFF
def convert_dicom_to_tiff(dicom_path, tiff_path, retries=5, delay=1):
    attempt = 0
    while attempt < retries:
        try:
            dicom_data = dcmread(dicom_path)
            pixel_data = dicom_data.pixel_array

            # Save as TIFF file using tifffile
            with tifffile.TiffWriter(tiff_path) as tiff:
                tiff.write(pixel_data, photometric='minisblack')
            print(f"Successfully converted {dicom_path} to {tiff_path}")
            return  # Exit function after successful conversion

        except PermissionError as e:
            print(f"Permission error on {dicom_path}: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
            attempt += 1
        except Exception as e:
            print(f"Failed to convert {dicom_path}: {e}.")
            return  # Exit function on other exceptions

    print(f"Failed to convert {dicom_path} after {retries} attempts.")

# GUIManager class handles updating the GUI and the plot
class GUIManager:
    def __init__(self, root, canvas, ax, event_handler):
        self.root = root
        self.canvas = canvas
        self.ax = ax
        self.event_handler = event_handler
        self.previous_blurred_images = set()
        self.previous_bad_images = set()

    def update_plot(self, mse_values, laplacian_vars, bad_indices, bad_images, blurred_images):
        self.ax.clear()
        
        # Plot MSE values
        if mse_values:
            self.ax.scatter(range(len(mse_values)), mse_values, color='b', label='MSE Values')
        
        # Plot bad MSE values in red
        if bad_indices:
            self.ax.scatter(bad_indices, [mse_values[i] for i in bad_indices], color='r', label='Bad Quality Images')
        
        self.ax.set_xlabel('Image Pair Index')
        self.ax.set_ylabel('Value')
        self.ax.set_title('Image Quality Analysis')
        self.ax.legend()
        self.ax.grid(True)
        
        # Draw threshold line
        self.ax.axhline(y=MSE_THRESHOLD, color='gray', linestyle='--', label='MSE Threshold')
        
        self.canvas.draw()  # Redraw the canvas

        # Update GUI messages
        self.update_blurred_images_message(blurred_images)
        self.update_mse_messages(mse_values, bad_indices)

    def update_blurred_images_message(self, blurred_images):
        new_blurred_images = set(blurred_images) - self.previous_blurred_images
        if new_blurred_images:
            message = "Blurred Images:\n" + "\n".join(new_blurred_images)
            blurred_images_label.config(text=message)
            self.previous_blurred_images.update(new_blurred_images)
        elif not blurred_images:
            blurred_images_label.config(text="No blurred images detected.")

    def update_mse_messages(self, mse_values, bad_indices):
        new_bad_images = {os.path.basename(self.event_handler.image_files[i + 1]) for i in bad_indices} - self.previous_bad_images
        if new_bad_images:
            message = "High MSE Images:\n" + "\n".join(f"Image Pair {i}: MSE={mse_values[i]}" for i in bad_indices)
            mse_label.config(text=message)
            self.previous_bad_images.update(new_bad_images)
        elif not bad_indices:
            mse_label.config(text="No high MSE images detected.")

# ImageHandler class monitors the folder for new images and triggers updates
class ImageHandler(FileSystemEventHandler):
    def __init__(self, gui_manager):
        super().__init__()
        self.image_files = []
        self.images = []
        self.mse_values = []
        self.laplacian_vars = []
        self.bad_indices = []
        self.bad_images = []
        self.blurred_images = []
        self.image_map = {}
        self.gui_manager = gui_manager

    def on_created(self, event):
        if event.is_directory:
            return
        
        filename = event.src_path

        # Handle DICOM files
        if filename.lower().endswith('.dcm'):
            tiff_path = filename.replace('.dcm', '.tif')
            convert_dicom_to_tiff(filename, tiff_path)
            filename = tiff_path  # Use TIFF path for further processing
        
            # Handle TIFF and other image files
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
            self.image_files.append(filename)
            self.image_files.sort()
                
            # Retry mechanism for file read
            for _ in range(10):
                try:
                    current_image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
                    if current_image is None:
                        raise ValueError("Image read returned None.")
                    print(f"Successfully read image {filename}")
                    break
                except (PermissionError, ValueError, cv2.error) as e:
                    print(f"Could not read image {filename}: {e}. Retrying...")
                    time.sleep(0.5)
            else:
                print(f"Failed to read image {filename} after multiple attempts. Skipping.")
                return
                
            self.image_files.append(filename)
            self.image_files.sort()
            self.image_map[filename] = current_image
            self.update_quality_metrics()

    def on_deleted(self, event):
        if event.is_directory:
            return
        
        filename = event.src_path
        if filename in self.image_map:
            del self.image_map[filename]
            self.update_quality_metrics()

    def update_quality_metrics(self):
        self.images = [self.image_map[f] for f in sorted(self.image_map.keys())]
        self.mse_values = []
        self.laplacian_vars = []
        self.bad_indices = []
        self.bad_images = []
        self.blurred_images = []

        if len(self.images) > 1:
            for i in range(len(self.images) - 1):
                mse = ImageProcessor.compute_mse(self.images[i], self.images[i + 1])
                self.mse_values.append(mse)
                if mse > MSE_THRESHOLD:
                    self.bad_indices.append(len(self.mse_values) - 1)
                    self.bad_images.append(os.path.basename(self.image_files[i + 1]))

            laplacian_var = ImageProcessor.compute_laplacian_variance(self.images[-1])
            self.laplacian_vars.append(laplacian_var)
            if laplacian_var < LAPLACIAN_VAR_THRESHOLD:
                self.blurred_images.append(os.path.basename(self.image_files[-1]))

        # Update the plot and messages in the GUI
        self.gui_manager.update_plot(self.mse_values, self.laplacian_vars, self.bad_indices, self.bad_images, self.blurred_images)


# Initialize the GUI and its widgets
class Application:
    def __init__(self, root, event_handler):
        self.root = root
        self.event_handler = event_handler
        self.mse_threshold = MSE_THRESHOLD
        self.laplacian_var_threshold = LAPLACIAN_VAR_THRESHOLD

        # GUI layout
        self.create_widgets()

    def create_widgets(self):
        global mse_label, blurred_images_label, threshold_entry, laplacian_threshold_entry

        # Folder path display and browse button
        self.folder_path_var = tk.StringVar(value=FOLDER_PATH)
        Label(self.root, text="Folder Path:").pack()
        self.folder_path_label = Label(self.root, textvariable=self.folder_path_var)
        self.folder_path_label.pack()
        Button(self.root, text="Browse", command=self.browse_folder).pack()

        # MSE threshold widgets
        Label(self.root, text=f"Current MSE Threshold: {self.mse_threshold}").pack()
        threshold_entry = Entry(self.root)
        threshold_entry.pack()
        Button(self.root, text="Set MSE Threshold", command=self.set_threshold).pack()

        # Laplacian variance threshold widgets
        Label(self.root, text=f"Current Laplacian Variance Threshold: {self.laplacian_var_threshold}").pack()
        laplacian_threshold_entry = Entry(self.root)
        laplacian_threshold_entry.pack()
        Button(self.root, text="Set Laplacian Threshold", command=self.set_laplacian_threshold).pack()

        # Create plot for MSE values
        fig = Figure(figsize=(10, 6), dpi=100)
        ax = fig.add_subplot(111)
        canvas = FigureCanvasTkAgg(fig, master=self.root)
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

        # Create labels for blurred images and MSE
        blurred_images_label = Label(self.root, text="No blurred images detected.", fg="red", bg="lightyellow", padx=10, pady=10, borderwidth=2, relief="solid")
        blurred_images_label.pack(pady=10)
        mse_label = Label(self.root, text="No high MSE images detected.", fg="red", bg="lightpink", padx=10, pady=10, borderwidth=2, relief="solid")
        mse_label.pack(pady=10)

        gui_manager = GUIManager(self.root, canvas, ax, self.event_handler)
        self.event_handler.gui_manager = gui_manager

    def browse_folder(self):
        folder_selected = filedialog.askdirectory(initialdir=FOLDER_PATH, title="Select Folder")
        if folder_selected:
            self.folder_path_var.set(folder_selected)
            self.event_handler.image_files.clear()  # Clear the current list of image files
            self.event_handler.image_map.clear()  # Clear the image map
            observer.schedule(self.event_handler, folder_selected, recursive=False)

    def set_threshold(self):
        global MSE_THRESHOLD
        try:
            new_threshold = float(threshold_entry.get())
            if new_threshold < 0:
                raise ValueError("Threshold must be non-negative.")
            MSE_THRESHOLD = new_threshold
        except ValueError as e:
            messagebox.showerror("Invalid Input", f"Please enter a valid number for the MSE threshold.\n{e}")

    def set_laplacian_threshold(self):
        global LAPLACIAN_VAR_THRESHOLD
        try:
            new_threshold = float(laplacian_threshold_entry.get())
            if new_threshold < 0:
                raise ValueError("Threshold must be non-negative.")
            LAPLACIAN_VAR_THRESHOLD = new_threshold
        except ValueError as e:
            messagebox.showerror("Invalid Input", f"Please enter a valid number for the Laplacian variance threshold.\n{e}")

if __name__ == "__main__":
    # Initialize the GUI and file system observer.
    root = tk.Tk()  # Creates the main window of the GUI
    root.title(config['application']['title'])  # Sets the title of the GUI Window
    root.geometry(f"{config['application']['window']['width']}x{config['application']['window']['height']}")  # Sets the size of GUI window (In pixels)

    image_handler = ImageHandler(None)  # Create instance of ImageHandler class
    app = Application(root, image_handler)

    # Starts the observer to monitor the specified folder for file changes.
    observer = Observer()  # Create instance of Observer from the watchdog library
    observer.schedule(image_handler, FOLDER_PATH, recursive=False)
    observer.start()

    # Callback function to be executed when the GUI window is closed (Stop the observer and quits the GUI application)
    root.protocol("WM_DELETE_WINDOW", lambda: observer.stop() or observer.join() or root.quit())
    root.mainloop()


Permission error on C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00000.dcm: [Errno 13] Permission denied: 'C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\\IMG00000.dcm'. Retrying in 1 seconds...


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Successfully converted C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00000.dcm to C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00000.tif
Successfully read image C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00000.tif
Successfully read image C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00000.tif


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Permission error on C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00001.dcm: [Errno 13] Permission denied: 'C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\\IMG00001.dcm'. Retrying in 1 seconds...
Successfully converted C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00001.dcm to C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00001.tif
Successfully read image C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00001.tif
Successfully read image C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00001.tif
Permission error on C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00002.dcm: [Errno 13] Permission denied: 'C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\\IMG00002.dcm'. Retrying in 1 seconds...
Successfully converted C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00002.dcm to C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/C

...

Permission error on C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00000.dcm: [Errno 13] Permission denied: 'C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\\IMG00000.dcm'. Retrying in 1 seconds...


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Successfully converted C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00000.dcm to C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00000.tif
Successfully read image C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00000.tif
Successfully read image C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00000.tif


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Permission error on C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00001.dcm: [Errno 13] Permission denied: 'C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\\IMG00001.dcm'. Retrying in 1 seconds...
Successfully converted C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00001.dcm to C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00001.tif
Successfully read image C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00001.tif
Successfully read image C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00001.tif
Permission error on C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00002.dcm: [Errno 13] Permission denied: 'C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\\IMG00002.dcm'. Retrying in 1 seconds...
Successfully converted C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/Current\IMG00002.dcm to C:/Users/ATripathi2/Oceaneering/2D_defect_segmentation/C

In [ ]:
import os
import time
import numpy as np
import cv2
import tifffile
import yaml
from pydicom import dcmread
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import tkinter as tk
from tkinter import Label, messagebox, Entry, Button, filedialog
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Load configuration from YAML file
with open("config.yml", "r") as config_file:
    config = yaml.safe_load(config_file)

# Global thresholds from config file
MSE_THRESHOLD = config['thresholds']['mse_threshold']
LAPLACIAN_VAR_THRESHOLD = config['thresholds']['laplacian_var_threshold']
FOLDER_PATH = config['application']['folder_path']

# ImageProcessor class handles MSE and Laplacian variance computations
class ImageProcessor:
    @staticmethod
    def compute_mse(image1, image2):
        if image1.shape != image2.shape:
            raise ValueError("Images must have the same dimensions for MSE computation.")
        mse = np.mean((image1 - image2) ** 2)
        return mse

    @staticmethod
    def compute_laplacian_variance(image):
        laplacian = cv2.Laplacian(image, cv2.CV_64F)
        variance = laplacian.var()
        return variance

# Function to convert DICOM to TIFF
def convert_dicom_to_tiff(dicom_path, tiff_path, retries=5, delay=1):
    attempt = 0
    while attempt < retries:
        try:
            dicom_data = dcmread(dicom_path)
            pixel_data = dicom_data.pixel_array

            # Save as TIFF file using tifffile
            with tifffile.TiffWriter(tiff_path) as tiff:
                tiff.write(pixel_data, photometric='minisblack')
            print(f"Successfully converted {dicom_path} to {tiff_path}")
            return  # Exit function after successful conversion

        except PermissionError as e:
            print(f"Permission error on {dicom_path}: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
            attempt += 1
        except Exception as e:
            print(f"Failed to convert {dicom_path}: {e}.")
            return  # Exit function on other exceptions

    print(f"Failed to convert {dicom_path} after {retries} attempts.")

# GUIManager class handles updating the GUI and the plot
class GUIManager:
    def __init__(self, root, canvas, ax, event_handler):
        self.root = root
        self.canvas = canvas
        self.ax = ax
        self.event_handler = event_handler
        self.previous_blurred_images = set()
        self.previous_bad_images = set()

    def update_plot(self, mse_values, laplacian_vars, bad_indices, bad_images, blurred_images):
        self.ax.clear()
        
        # Plot MSE values
        if mse_values:
            self.ax.scatter(range(len(mse_values)), mse_values, color='b', label='MSE Values')
        
        # Plot bad MSE values in red
        if bad_indices:
            self.ax.scatter(bad_indices, [mse_values[i] for i in bad_indices], color='r', label='Bad Quality Images')
        
        self.ax.set_xlabel('Image Pair Index')
        self.ax.set_ylabel('Value')
        self.ax.set_title('Image Quality Analysis')
        self.ax.legend()
        self.ax.grid(True)
        
        # Draw threshold line
        self.ax.axhline(y=MSE_THRESHOLD, color='gray', linestyle='--', label='MSE Threshold')
        
        self.canvas.draw()  # Redraw the canvas

        # Update GUI messages
        self.update_blurred_images_message(blurred_images)
        self.update_mse_messages(mse_values, bad_indices)

    def update_blurred_images_message(self, blurred_images):
        new_blurred_images = set(blurred_images) - self.previous_blurred_images
        if new_blurred_images:
            message = "Blurred Images:\n" + "\n".join(new_blurred_images)
            blurred_images_label.config(text=message)
            self.previous_blurred_images.update(new_blurred_images)
        elif not blurred_images:
            blurred_images_label.config(text="No blurred images detected.")

    def update_mse_messages(self, mse_values, bad_indices):
        new_bad_images = {os.path.basename(self.event_handler.image_files[i + 1]) for i in bad_indices} - self.previous_bad_images
        if new_bad_images:
            message = "High MSE Images:\n" + "\n".join(f"Image Pair {i}: MSE={mse_values[i]}" for i in bad_indices)
            mse_label.config(text=message)
            self.previous_bad_images.update(new_bad_images)
        elif not bad_indices:
            mse_label.config(text="No high MSE images detected.")

# ImageHandler class monitors the folder for new images and triggers updates
class ImageHandler(FileSystemEventHandler):
    def __init__(self, gui_manager):
        super().__init__()
        self.image_files = []
        self.images = []
        self.mse_values = []
        self.laplacian_vars = []
        self.bad_indices = []
        self.bad_images = []
        self.blurred_images = []
        self.image_map = {}
        self.gui_manager = gui_manager

    def on_created(self, event):
        if event.is_directory:
            return
        
        filename = event.src_path

        # Handle DICOM files
        if filename.lower().endswith('.DCM'):
            tiff_path = filename.replace('.DCM', '.tif')
            convert_dicom_to_tiff(filename, tiff_path)
            filename = tiff_path  # Use TIFF path for further processing
        
            # Handle TIFF and other image files
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
            self.image_files.append(filename)
            self.image_files.sort()
                
            # Retry mechanism for file read
            for _ in range(10):
                try:
                    current_image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
                    if current_image is None:
                        raise ValueError("Image read returned None.")
                    print(f"Successfully read image {filename}")
                    break
                except (PermissionError, ValueError, cv2.error) as e:
                    print(f"Could not read image {filename}: {e}. Retrying...")
                    time.sleep(0.5)
            else:
                print(f"Failed to read image {filename} after multiple attempts. Skipping.")
                return
                
            self.image_files.append(filename)
            self.image_files.sort()
            self.image_map[filename] = current_image
            self.update_quality_metrics()


    def on_deleted(self, event):
        if event.is_directory:
            return
        
        filename = event.src_path
        if filename in self.image_map:
            del self.image_map[filename]
            self.update_quality_metrics()

    def update_quality_metrics(self):
        self.images = [self.image_map[f] for f in sorted(self.image_map.keys())]
        self.mse_values = []
        self.laplacian_vars = []
        self.bad_indices = []
        self.bad_images = []
        self.blurred_images = []

        if len(self.images) > 1:
            for i in range(len(self.images) - 1):
                mse = ImageProcessor.compute_mse(self.images[i], self.images[i + 1])
                self.mse_values.append(mse)
                if mse > MSE_THRESHOLD:
                    self.bad_indices.append(len(self.mse_values) - 1)
                    self.bad_images.append(os.path.basename(self.image_files[i + 1]))

            laplacian_var = ImageProcessor.compute_laplacian_variance(self.images[-1])
            self.laplacian_vars.append(laplacian_var)
            if laplacian_var < LAPLACIAN_VAR_THRESHOLD:
                self.blurred_images.append(os.path.basename(self.image_files[-1]))

        # Update the plot and messages in the GUI
        self.gui_manager.update_plot(self.mse_values, self.laplacian_vars, self.bad_indices, self.bad_images, self.blurred_images)


# Initialize the GUI and its widgets
class Application:
    def __init__(self, root, event_handler):
        self.root = root
        self.event_handler = event_handler
        self.mse_threshold = MSE_THRESHOLD
        self.laplacian_var_threshold = LAPLACIAN_VAR_THRESHOLD

        # GUI layout
        self.create_widgets()

    def create_widgets(self):
        global mse_label, blurred_images_label, threshold_entry, laplacian_threshold_entry

        # Folder path display and browse button
        self.folder_path_var = tk.StringVar(value=FOLDER_PATH)
        Label(self.root, text="Folder Path:").pack()
        self.folder_path_label = Label(self.root, textvariable=self.folder_path_var)
        self.folder_path_label.pack()
        Button(self.root, text="Browse", command=self.browse_folder).pack()

        # MSE threshold widgets
        Label(self.root, text=f"Current MSE Threshold: {self.mse_threshold}").pack()
        threshold_entry = Entry(self.root)
        threshold_entry.pack()
        Button(self.root, text="Set MSE Threshold", command=self.set_threshold).pack()

        # Laplacian variance threshold widgets
        Label(self.root, text=f"Current Laplacian Variance Threshold: {self.laplacian_var_threshold}").pack()
        laplacian_threshold_entry = Entry(self.root)
        laplacian_threshold_entry.pack()
        Button(self.root, text="Set Laplacian Threshold", command=self.set_laplacian_threshold).pack()

        # Create plot for MSE values
        fig = Figure(figsize=(10, 6), dpi=100)
        ax = fig.add_subplot(111)
        canvas = FigureCanvasTkAgg(fig, master=self.root)
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

        # Create labels for blurred images and MSE
        blurred_images_label = Label(self.root, text="No blurred images detected.", fg="red", bg="lightyellow", padx=10, pady=10, borderwidth=2, relief="solid")
        blurred_images_label.pack(pady=10)
        mse_label = Label(self.root, text="No high MSE images detected.", fg="red", bg="lightpink", padx=10, pady=10, borderwidth=2, relief="solid")
        mse_label.pack(pady=10)

        gui_manager = GUIManager(self.root, canvas, ax, self.event_handler)
        self.event_handler.gui_manager = gui_manager

    def browse_folder(self):
        folder_selected = filedialog.askdirectory(initialdir=FOLDER_PATH, title="Select Folder")
        if folder_selected:
            self.folder_path_var.set(folder_selected)
            self.event_handler.image_files.clear()  # Clear the current list of image files
            self.event_handler.image_map.clear()  # Clear the image map
            observer.schedule(self.event_handler, folder_selected, recursive=False)

    def set_threshold(self):
        global MSE_THRESHOLD
        try:
            new_threshold = float(threshold_entry.get())
            if new_threshold < 0:
                raise ValueError("Threshold must be non-negative.")
            MSE_THRESHOLD = new_threshold
        except ValueError as e:
            messagebox.showerror("Invalid Input", f"Please enter a valid number for the MSE threshold.\n{e}")

    def set_laplacian_threshold(self):
        global LAPLACIAN_VAR_THRESHOLD
        try:
            new_threshold = float(laplacian_threshold_entry.get())
            if new_threshold < 0:
                raise ValueError("Threshold must be non-negative.")
            LAPLACIAN_VAR_THRESHOLD = new_threshold
        except ValueError as e:
            messagebox.showerror("Invalid Input", f"Please enter a valid number for the Laplacian variance threshold.\n{e}")

if __name__ == "__main__":
    # Initialize the GUI and file system observer.
    root = tk.Tk()  # Creates the main window of the GUI
    root.title(config['application']['title'])  # Sets the title of the GUI Window
    root.geometry(f"{config['application']['window']['width']}x{config['application']['window']['height']}")  # Sets the size of GUI window (In pixels)

    image_handler = ImageHandler(None)  # Create instance of ImageHandler class
    app = Application(root, image_handler)

    # Starts the observer to monitor the specified folder for file changes.
    observer = Observer()  # Create instance of Observer from the watchdog library
    observer.schedule(image_handler, FOLDER_PATH, recursive=False)
    observer.start()

    # Callback function to be executed when the GUI window is closed (Stop the observer and quits the GUI application)
    root.protocol("WM_DELETE_WINDOW", lambda: observer.stop() or observer.join() or root.quit())
    root.mainloop()


C:\Users\ATripathi2\AppData\Local\anaconda3\envs\open3d_env\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ATripathi2\AppData\Local\anaconda3\envs\open3d_env\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\ATripathi2\AppData\Local\anaconda3\envs\open3d_env\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
